In [1]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os
import re
from typing import Any, List
import glob
import matplotlib.pyplot as plt
import json
import random


backgrounds = glob.glob("resources/paper/*.jpg")
count = 100
textFile = open("resources/corpus/sa.txt", "r")
text = textFile.read()
textFile.close()
ALLIGNMENT_LEFT = 0
ALLIGNMENT_CENTER = 1 
ALLIGNMENT_RIGHT = 2
ALLIGNMENT_TOP = 3
ALLIGNMENT_BOTTOM = 4
def text_box(text, image_draw, font, box, horizontal_allignment = ALLIGNMENT_LEFT, vertical_allignment = ALLIGNMENT_TOP, **kwargs):
    x = box[0]
    y = box[1]
    width = box[2]
    height = box[3]
    lines = text.split('\n')
    true_lines = []
    for line in lines:
        if font.getsize(line)[0] <= width:
            true_lines.append(line) 
        else:
            current_line = ''
            for word in line.split(' '):
                if font.getsize(current_line + word)[0] <= width:
                    current_line += ' ' + word 
                else:
                    true_lines.append(current_line)
                    current_line = word 
            true_lines.append(current_line)
    
    x_offset = y_offset = 0
    lineheight = font.getsize(true_lines[0])[1] * 1.2 # Give a margin of 0.2x the font height
    if vertical_allignment == ALLIGNMENT_CENTER:
        y = int(y + height / 2)
        y_offset = - (len(true_lines) * lineheight) / 2
    elif vertical_allignment == ALLIGNMENT_BOTTOM:
        y = int(y + height)
        y_offset = - (len(true_lines) * lineheight)
    
    for line in true_lines:
        linewidth = font.getsize(line)[0]
        if horizontal_allignment == ALLIGNMENT_CENTER:
            x_offset = (width - linewidth) / 2
        elif horizontal_allignment == ALLIGNMENT_RIGHT:
            x_offset = width - linewidth
        image_draw.text(
            (int(x + x_offset), int(y + y_offset)),
            line,
            font=font,
            **kwargs
        )
        y_offset += lineheight
    


def getBoxesFromImage(image, boxes,text, fontSize):
    #genrate new boxes till they are not within 200 pixels of any other box
    newBoxes = []
    #check if sanskrit in text
    


    h = len(text) * 4
    w = len(text) * 2
    sansText = re.findall(r'[\u0900-\u097F]+', text)
    if len(sansText) > 0:
        w = len(sansText) * 10
    x1 = np.random.randint(0, image.size[0] - w)
    y1 = np.random.randint(0, image.size[1] - h)
    x2 = x1 + w
    y2 = y1 + h
    box = (x1, y1, x2, y2)
    newBoxes.append(box)
    for b in boxes:
        if abs(b[0] - x1) < 250 and abs(b[1] - y1) < 400 and abs(b[2] - x2) < 250 and abs(b[3] - y2) < 400:
            return getBoxesFromImage(image, boxes, text, fontSize)
    
    return newBoxes + boxes


def setTextOnImage(texts, background):
    newTempIamge = Image.open(background)
    bounds = []
    random_font = np.random.randint(40, 80)
    h = newTempIamge.size[1]
    w = newTempIamge.size[0]

    font = ImageFont.truetype("resources/font/sa/TiroDevanagariSanskrit-Regular.ttf", random_font)
    draw = ImageDraw.Draw(newTempIamge)
    #join texts with delimeter
    text = "\n".join(texts)
    text_box(text, draw, font, (0, 0, w, h), stroke_fill=(0, 0, 0), horizontal_allignment = ALLIGNMENT_LEFT, vertical_allignment = ALLIGNMENT_TOP,fill="#000")
    return newTempIamge
    
    

# def setTextOnImage(texts, background):
#     newTempIamge = Image.open(background)
#     bounds = []
#     random_font = np.random.randint(40, 80)

    
#     print(bounds)
    
    
#     l = 0
#     for text in texts:
#         l += 1
#         #make new box till it is not within 200 pixels of any other box
#         font = ImageFont.truetype("resources/font/sa/TiroDevanagariSanskrit-Italic.ttf", random_font)
#         if l == l%2:
#             font = ImageFont.truetype("resources/font/sa/TiroDevanagariSanskrit-Regular.ttf", random_font)

#         draw = ImageDraw.Draw(newTempIamge)
#         devnagriDelimeter = "।"
#         englishDelimeter = "."
        

#         i=0
#         print(len(text.split("#")))
#         for t in text.split("#"):
#             print(t)
#             bounds = getBoxesFromImage(newTempIamge, bounds, text, random_font)
#             text_box(t, draw, font, bounds[i], stroke_fill=(0, 0, 0), horizontal_allignment = ALLIGNMENT_LEFT, vertical_allignment = ALLIGNMENT_TOP)
#             i+=1
        
        
#         #dwaw text in multiple lines
        
        
#         # print(box, text)
    
#     #lower the quality of image to simulate grany text on paper
    
#     return newTempIamge

def convertTextIntoParagraphs(text):
    #convert text into sections till 10 new lines are found
    textSections = []
    tempText = text.split("\n")
    tempText = [x for x in tempText if x != '']
    #take only 20 words from each section
    texts = []
    for t in tempText:
        texts.append(" ".join(t.split(" ")[:20]))
    tempText = texts
    #merege 10 lines into one section
    for i in range(0, len(tempText), 1):
        textSections.append("#".join((tempText[i:i+1])))
    return textSections
    
#{"file_name": "image_0.jpg", "ground_truth": "{\"gt_parse\": {\"text_sequence\": \"\\\\u0938\\\\u0924\\\\u094d\\\\u0915\\\\u0943\\\\u0924\\\\u094b \\\\u0930\\\\u093e\\\\u092e\\\\u0903 \\\\u0938\\\\u092d\\\\u093e\\\\u0930\\\\u094d\\\\u092f\\\\u0903 \\\\u0938\\\\u0939\\\\u0932\\\\u0915\\\\u094d\\\\u0937 \\\\u094d\\\\u092e\\\\u0923\\\\u0903\\\\u0965 By luck i t is that afte r a long ti me, R\\\\u0101ma h as come to see me. I had me ntally wished for his arrival. Go you; and let R\\\\u0101ma, having been res pectfully received, come before me. Why have you not brought h imself?\\\\u092a\\\\u094d\\\\u0930\\\\u0935\\\\u0947\\\\u0938\\\\u094d\\\\u092f\\\\u0924 \\\\u093e\\\\u0902 \\\\u0938\\\\u092e\\\\u0940\\\\u092a\\\\u0902 \\\\u092e\\\\u0947 \\\\u0915\\\\u093f \\\\u092e\\\\u0938\\\\u094c \\\\u0928 \\\\u092a\\\\u094d\\\\u0930\\\\u0935\\\\u0947\\\\u0936 \\\\u093f\\\\u0924\\\\u0903\\\\u0964 \\\\u090f\\\\u0935\\\\u092e\\\\u0941\\\\u0915\\\\u094d\\\\u0924\\\\u0938\\\\u094d\\\\u0924\\\\u0941 \\\\u092e\\\\u0941\\\\u0928\\\\u093f\\\\u0928\\\\u093e \\\\u0927 \\\\u0930\\\\u094d\\\\u092e\\\\u091c\\\\u094d\\\\u091e\\\\u0947\\\\u0928 \\\\u092e\\\\u0939\\\\u093e\\\\u0924\\\\u094d\\\\u092e \\\\u0928\\\\u093e\\\\u0965 \\\\u0905\\\\u092d\\\\u093f\\\\u0935\\\\u093e\\\\u0926\\\\u094d\\\\u092f \\\\u093e\\\\u092c\\\\u094d\\\\u0930\\\\u0935\\\\u0940\\\\u091a \\\\u094d\\\\u091b\\\\u093f\\\\u0937\\\\u094d\\\\u092f \\\\u0938\\\\u094d\\\\u0924\\\\u0925 \\\\u0947\\\\u0924\\\\u093f \\\\u0928 \\\\u093f\\\\u092f\\\\u0924\\\\u093e\\\\u091e \\\\u094d\\\\u091c\\\\u0932\\\\u093f\\\\u0903\\\\u0964T hus addr essed by t he high- souled an d righteous asce tic, the disci ple salut ing h im, wit h joi ned hand s, sa id, S o be it.\\\\u0924 \\\\u0926\\\\u093e \\\\u0928\\\\u093f\\\\u0937 \\\\u094d\\\\u0915\\\\u094d\\\\u0930\\\\u092e \\\\u094d\\\\u092f \\\\u0938 \\\\u092e\\\\u094d\\\\u092d \\\\u094d\\\\u0930\\\\u093e \\\\u0928\\\\u094d\\\\u0924\\\\u0903 \\\\u0936\\\\u093f\\\\u0937\\\\u094d\\\\u092f \\\\u094b \\\\u0932\\\\u0915 \\\\u094d\\\\u0937\\\\u094d\\\\u092e \\\\u0923\\\\u092e\\\\u092c\\\\u094d\\\\u0930 \\\\u0935\\\\u0940\\\\u0924\\\\u094d \\\\u0965 \\\\u0915\\\\u094b\\\\u093d\"}}"}

def main():
    textSections = convertTextIntoParagraphs(text)
    # print(textSections[:100])
    cuurentTextSection = 0
    allMetaData = []
    c= 1
    val = "Train"
    for x in range(0,10000):
        #set train and test and val
        if c > 800:
            val = "Test"
        if c > 900:
            val = "Val"
        #random select backround
        background = random.choice(backgrounds)

        c +=1
        #pick text selection in gap of 4
        texts = textSections[cuurentTextSection:cuurentTextSection+15]
        cuurentTextSection = cuurentTextSection + 15
        joinedText = " ".join(texts)
        joinedText = joinedText.replace("#", " ")


        #set text on image
        newTempIamge = setTextOnImage(texts, background)
        #save image
        newTempIamge.save(f"outputs/SynthDoG_sa/{val}/" + str(x) + ".jpg")
        #save metadata
        # print(joinedText)
        metadata = {"file_name": str(c) + ".jpg", "ground_truth": {
            "gt_parse": {
                "text_sequence": joinedText

            }
        }}
        allMetaData.append(metadata)

        
        #save metadata
    with open(f'outputs/SynthDoG_sa/{val}/metadata.jsonl', 'w') as outfile:
        json.dump(allMetaData, outfile, ensure_ascii=False)

main()

    

        



/tmp/ipykernel_14051/1978137298.py:30: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line)[0] <= width:
/tmp/ipykernel_14051/1978137298.py:35: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(current_line + word)[0] <= width:
/tmp/ipykernel_14051/1978137298.py:43: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  lineheight = font.getsize(true_lines[0])[1] * 1.2 # Give a margin of 0.2x the font height
/tmp/ipykernel_14051/1978137298.py:52: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  linewidth = font.getsize(line)[0]


FileNotFoundError: [Errno 2] No such file or directory: 'SynthDoG_sa/outputs/Train/0.jpg'